In [18]:
import torch

import torch.nn as nn
import os
import shutil
import itertools
import random

from chofer_torchex.utils.data.collate import dict_sample_target_iter_concat
from chofer_torchex.utils.functional import collection_cascade, cuda_cascade
from jmlr_2018_code.datasets import Mpeg7
from jmlr_2018_code.utils import *
from chofer_torchex.nn.slayer import SLayerExponential, SLayerRational, LogStretchedBirthLifeTimeCoordinateTransform, prepare_batch, SLayerRationalHat
from sklearn.model_selection import ShuffleSplit
from collections import Counter, defaultdict
from torch.utils.data import DataLoader
from collections import OrderedDict
from torch.autograd import Variable

%matplotlib notebook
%load_ext autoreload
%autoreload 2

os.environ['CUDA_VISIBLE_DEVICES'] = str(0)


class train_env:
    nu = 0.01
    n_epochs = 600
    lr_initial = 0.01
    momentum = 0.9
    lr_epoch_step = 100
    batch_size = 100
    train_size = 0.9
    

coordinate_transform = LogStretchedBirthLifeTimeCoordinateTransform(nu=train_env.nu)
            

used_directions = ['dim_0_dir_{}'.format(i) for i in range(0, 32,2)]
mpeg7_data_set = Mpeg7(root_dir='./data')
mpeg7_data_set.sample_transforms = [
                                    lambda x: {k: x[k] for k in used_directions}, 
                                    numpy_to_torch_cascade,
                                    lambda x: collection_cascade(x, 
                                                                 lambda x: isinstance(x, torch._TensorBase), 
                                                                 lambda x: coordinate_transform(x))
                                   ]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found data!


In [19]:
class PHTCollate:   
    def __init__(self, nu, cuda=True, rotation_augmentation=False):
        self.cuda = cuda
        self.rotation_augmentation = rotation_augmentation
        
    def __call__(self, sample_target_iter):
        
        augmented_samples = []
        if self.rotation_augmentation:
            samples, targets = [], []
            for x, y in sample_target_iter:                
                i = random.randint(0, len(used_directions)-1)
                shifted_keys = used_directions[i:] + used_directions[:i]                
                
                samples.append({k: x[ki] for k, ki in zip(used_directions, shifted_keys)})
                targets.append(y)
                
            sample_target_iter = zip(samples, targets)

        x, y = dict_sample_target_iter_concat(sample_target_iter)                                            
                                              
        for k in x.keys():
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 2)                  

        y = torch.LongTensor(y)    

        if self.cuda:
            # Shifting the necessary parts of the prepared batch to the cuda
            x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                 for k, v in x.items()}

            y = y.cuda()

        return x, y                       
    
collate_fn_train = PHTCollate(train_env.nu, cuda=True)
collate_fn_test = PHTCollate(train_env.nu, cuda=True)

In [20]:
train_sampler, test_sampler = get_train_test_sampler(mpeg7_data_set, train_env.train_size, stratified=True)

dl_train = DataLoader(mpeg7_data_set,
                      batch_size=train_env.batch_size, 
                      collate_fn=collate_fn_train,
                      sampler=train_sampler)

dl_test = DataLoader(mpeg7_data_set,
                     batch_size=train_env.batch_size, 
                     collate_fn=collate_fn_test, 
                     sampler=test_sampler)

Generated training and testing split:
Train: Counter({44: 18, 57: 18, 58: 18, 60: 18, 47: 18, 19: 18, 25: 18, 21: 18, 64: 18, 68: 18, 43: 18, 8: 18, 15: 18, 23: 18, 38: 18, 16: 18, 12: 18, 31: 18, 32: 18, 9: 18, 24: 18, 14: 18, 22: 18, 18: 18, 6: 18, 66: 18, 0: 18, 50: 18, 1: 18, 48: 18, 27: 18, 39: 18, 45: 18, 53: 18, 11: 18, 59: 18, 56: 18, 28: 18, 42: 18, 35: 18, 51: 18, 26: 18, 46: 18, 67: 18, 41: 18, 54: 18, 4: 18, 49: 18, 34: 18, 5: 18, 13: 18, 63: 18, 40: 18, 65: 18, 61: 18, 69: 18, 10: 18, 62: 18, 52: 18, 33: 18, 7: 18, 2: 18, 30: 18, 36: 18, 3: 18, 37: 18, 17: 18, 29: 18, 55: 18, 20: 18})
Test: Counter({59: 2, 47: 2, 67: 2, 29: 2, 1: 2, 56: 2, 19: 2, 34: 2, 36: 2, 6: 2, 46: 2, 60: 2, 25: 2, 53: 2, 43: 2, 37: 2, 18: 2, 31: 2, 14: 2, 64: 2, 39: 2, 68: 2, 26: 2, 7: 2, 63: 2, 10: 2, 48: 2, 69: 2, 9: 2, 66: 2, 50: 2, 2: 2, 52: 2, 55: 2, 17: 2, 42: 2, 33: 2, 5: 2, 22: 2, 40: 2, 4: 2, 35: 2, 61: 2, 0: 2, 16: 2, 51: 2, 32: 2, 21: 2, 23: 2, 15: 2, 38: 2, 49: 2, 57: 2, 44: 2, 45: 2, 28:

In [ ]:
def Slayer(n_elements):
    return SLayerRationalHat(n_elements, radius_init=0.1)
#     return SLayerRational(n_elements=n_elements, 
#                           point_dimension=2, 
#                           sharpness_init=25, 
#                           exponent_init=1, 
#                           share_sharpness=False,
#                           share_exponent=False,
#                           pointwise_activation_threshold=None,
#                           freeze_exponent=False
#                           )

#     return SLayerExponential(n_elements=n_elements, point_dimension=2)


def LinearCell(n_in, n_out):
    m = nn.Sequential(nn.Linear(n_in, n_out), 
                      nn.BatchNorm1d(n_out), 
                      nn.ELU()
                     )
    m.out_features = m[0].out_features
    return m


class Mpeg7_model(nn.Module):
    def __init__(self):
        super().__init__()   
        self.n_elements = 150
        
        self.reccurent_in_dim = self.n_elements #int(self.n_elements/2)
        self.reccurent_out_dim = self.n_elements #int(self.n_elements/2)
        self.batch_size = train_env.batch_size
        
        self.slayers = ModuleDict()
#         self.slayers_2_recurrent_linears = ModuleDict()
        for k in used_directions:
            s = Slayer(self.n_elements)
            self.slayers[k] = s            
#             l = nn.Linear(s.n_elements, self.reccurent_in_dim)
#             self.slayers_2_recurrent_linears[k] = LinearCell(self.n_elements, self.reccurent_in_dim)
            
        self.recurrent = nn.LSTM(self.reccurent_in_dim, self.reccurent_out_dim)
        
        n_1 =1000
        self.cls = nn.Sequential(
                                 nn.Dropout(0.4),
                                 nn.BatchNorm1d(self.reccurent_out_dim),
                                 LinearCell(self.reccurent_out_dim, n_1),
                                 nn.Dropout(0.3),
                                 LinearCell(n_1, int(n_1/2)),
                                 nn.Dropout(0.2),
                                 nn.Linear(int(n_1/2), 70)
                                )
        
    def forward(self, input):
        x = []
        for k in used_directions:            
            xx = self.slayers[k](input[k])
#             xx = self.slayers_2_recurrent_linears[k](xx)
            x.append(xx)

        x = torch.stack(x, dim=0)  
        x = torch.cat([x]*10, dim=0)
        
        x, _ = self.recurrent(x)
        
        x = x[-1]
        x = x.squeeze()
        
        x = self.cls(x)        
               
        return x
    
    def parameters_split(self):
        return {'non_linear': self.slayers.parameters(),
                'linear': itertools.chain(self.cls.parameters(), self.recurrent.parameters())}
    
    def center_init(self, sample_target_iter):
        centers = k_means_center_init(sample_target_iter, self.n_elements)
        
        for k, v in centers.items():
            self.slayers._modules[k].centers.data = v
            
            

In [ ]:
model = Mpeg7_model()
model.center_init([mpeg7_data_set[i] for i in train_sampler])
model.cuda()

stats = defaultdict(list)
opt = torch.optim.SGD(model.parameters(), lr=train_env.lr_initial, momentum=train_env.momentum)

for i_epoch in range(1, train_env.n_epochs+1):      
    
    model.train()
        
    epoch_loss = 0    
    
    if i_epoch % train_env.lr_epoch_step == 0:
        adapt_lr(opt, lambda lr: lr*0.5)
    
    for i_batch, (x, y) in enumerate(dl_train, 1):              
        
        y = torch.autograd.Variable(y)
        
        def closure():
            opt.zero_grad()
            y_hat = model(x)            
            loss = nn.functional.cross_entropy(y_hat, y)   
            loss.backward()
            
#             torch.nn.utils.clip_grad_norm(model.parameters_split()['non_linear'], 1, norm_type='inf')
            return loss

        loss = opt.step(closure)
        
        epoch_loss += float(loss)
        stats['loss_by_batch'].append(float(loss))
        stats['centers'].append(model.slayers['dim_0_dir_0'].centers.data.cpu().numpy())
        
        print("Epoch {}/{}, Batch {}/{}".format(i_epoch, train_env.n_epochs, i_batch, len(dl_train)), end="       \r")
  
    stats['train_loss_by_epoch'].append(epoch_loss/len(dl_train))
        
    print("\n\r testing...")
    model.eval()    
    true_samples = 0
    seen_samples = 0
    epoch_test_loss = 0
    for i_batch, (x, y) in enumerate(dl_test):

        y_hat = model(x)
        epoch_test_loss += float(nn.functional.cross_entropy(y_hat, torch.autograd.Variable(y.cuda())).data)

        y_hat = y_hat.max(dim=1)[1].data.long()

        true_samples += (y_hat == y).sum()
        seen_samples += y.size(0)  
     
    stats['test_accuracy'].append(true_samples/seen_samples)
    stats['test_loss_by_epoch'].append(epoch_test_loss/len(dl_test))
    print(true_samples/seen_samples)

Epoch 1/600, Batch 13/13       
 testing...
0.02142857142857143
Epoch 2/600, Batch 13/13       
 testing...
0.07142857142857142
Epoch 3/600, Batch 13/13       
 testing...
0.07857142857142857
Epoch 4/600, Batch 13/13       
 testing...
0.14285714285714285
Epoch 5/600, Batch 13/13       
 testing...
0.24285714285714285
Epoch 6/600, Batch 13/13       
 testing...
0.2857142857142857
Epoch 7/600, Batch 13/13       
 testing...
0.37142857142857144
Epoch 8/600, Batch 13/13       
 testing...
0.45
Epoch 9/600, Batch 13/13       
 testing...
0.39285714285714285
Epoch 10/600, Batch 13/13       
 testing...
0.35
Epoch 11/600, Batch 13/13       
 testing...
0.5071428571428571
Epoch 12/600, Batch 13/13       
 testing...
0.4928571428571429
Epoch 13/600, Batch 13/13       
 testing...
0.6071428571428571
Epoch 14/600, Batch 13/13       
 testing...
0.5071428571428571
Epoch 15/600, Batch 13/13       
 testing...
0.5142857142857142
Epoch 16/600, Batch 13/13       
 testing...
0.5
Epoch 17/600, Batch 1

Epoch 131/600, Batch 13/13       
 testing...
0.8785714285714286
Epoch 132/600, Batch 13/13       
 testing...
0.8785714285714286
Epoch 133/600, Batch 13/13       
 testing...
0.8928571428571429
Epoch 134/600, Batch 13/13       
 testing...
0.8928571428571429
Epoch 135/600, Batch 13/13       
 testing...
0.8928571428571429
Epoch 136/600, Batch 13/13       
 testing...
0.8857142857142857
Epoch 137/600, Batch 13/13       
 testing...
0.8571428571428571
Epoch 138/600, Batch 13/13       
 testing...
0.8857142857142857
Epoch 139/600, Batch 13/13       
 testing...
0.8857142857142857
Epoch 140/600, Batch 13/13       
 testing...
0.8785714285714286
Epoch 141/600, Batch 13/13       
 testing...
0.85
Epoch 142/600, Batch 13/13       
 testing...
0.8285714285714286
Epoch 143/600, Batch 13/13       
 testing...
0.85
Epoch 144/600, Batch 13/13       
 testing...
0.8642857142857143
Epoch 145/600, Batch 13/13       
 testing...
0.8785714285714286
Epoch 146/600, Batch 13/13       
 testing...
0.85714

In [ ]:
plt.figure()

if 'centers' in stats:
    c_start = stats['centers'][0]
    c_end = stats['centers'][-1]

    plt.plot(c_start[:,0], c_start[:, 1], 'bo', label='center initialization')
    plt.plot(c_end[:,0], c_end[:, 1], 'ro', label='center learned')

    all_centers = numpy.stack(stats['centers'], axis=0)
    for i in range(all_centers.shape[1]):
        points = all_centers[:,i, :]
        plt.plot(points[:, 0], points[:, 1], '-k', alpha=0.25)
        

    plt.legend()
    
plt.figure()
plt.plot(stats['train_loss_by_epoch'], label='train_loss')
plt.plot(stats['test_loss_by_epoch'], label='test_loss')
plt.plot(stats['test_accuracy'], label='test_accuracy')


plt.legend()
plt.show()

In [ ]:
stats['test_accuracy'][-20:]

In [ ]:
model.slayers['dim_0_dir_0'].radius.min()